# NVDA–SPX Risk & Dependence (Step-by-step)

We’ll build this project module-by-module.

Next: implement `src/nvda_spx/data.py` to download prices and compute returns.


In [0]:
# We'll start writing code here after implementing src/nvda_spx/data.py
